In [32]:
from transformers import AutoTokenizer, AutoModel
from transformers import GPT2Model, GPT2Config, GPT2Tokenizer
from transformers import BertModel, BertTokenizer
import torch
import json
import onnx
import plotly.io as pio
import plotly.express as px
import pandas as pd
import numpy as np
# Visualize the embeddings using PCA
from sklearn.decomposition import PCA

# Specify the path to the kaleido runtime executable
# Replace '/path/to/kaleido' with the actual path to the kaleido executable
# pio.kaleido.scope.path = 'C:\1\a\Lib\site-packages\kaleido\executable\bin'

In [ ]:
# Load the model and tokenizer
model_name = "bert-base-uncased"
model = AutoModel.from_pretrained(model_name, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
# # Load the model and tokenizer
# model_name = "bert-base-uncased"
# model = BertModel.from_pretrained(model_name, output_attentions=True)
# tokenizer = BertTokenizer.from_pretrained(model_name)

In [4]:
# Ensure the model is on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
o = model.to(device)

In [5]:
o

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [6]:
model.__repr__()

'BertModel(\n  (embeddings): BertEmbeddings(\n    (word_embeddings): Embedding(30522, 768, padding_idx=0)\n    (position_embeddings): Embedding(512, 768)\n    (token_type_embeddings): Embedding(2, 768)\n    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)\n    (dropout): Dropout(p=0.1, inplace=False)\n  )\n  (encoder): BertEncoder(\n    (layer): ModuleList(\n      (0-11): 12 x BertLayer(\n        (attention): BertAttention(\n          (self): BertSdpaSelfAttention(\n            (query): Linear(in_features=768, out_features=768, bias=True)\n            (key): Linear(in_features=768, out_features=768, bias=True)\n            (value): Linear(in_features=768, out_features=768, bias=True)\n            (dropout): Dropout(p=0.1, inplace=False)\n          )\n          (output): BertSelfOutput(\n            (dense): Linear(in_features=768, out_features=768, bias=True)\n            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)\n            (dropout): Dropou

In [7]:
model.__dict__

{'training': False,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict([('embeddings',
               BertEmbeddings(
                 (word_embeddings): Embedding(30522, 768, padding_idx=0)
                 (position_embeddings): Embedding(512, 768)
                 (token_type_embeddings): Embedding(2, 768)
                 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                 (dropout): Dropout(p=0

In [8]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_assisted_decoding',
 '_auto_class',
 '_autoset_attn_implementation',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_beam_search',
 '_buffers',
 '_call_impl',
 '_check_and_enable_flash_attn_2',
 '_check_and_enable_sdpa',
 '_compiled_call_impl',
 '_constrained_beam_search',
 '_contrastive_search',
 '_convert_head_mask_to_5d',
 '_copy_lm_head_original_to_resized',
 '_create_repo',
 '_dispatch_accelerate_model',
 '_dola_decoding',
 '_expand_inputs_for_generation',

In [9]:
# Get the model's dictionary
model_dict = model.__dict__
model_str = model.__repr__()

In [10]:
# Get the model's architecture
model_config = model.config.to_dict()

In [11]:
def convert_to_json_serializable(obj):
    if isinstance(obj, (int, float, bool, str, type(None))):
        return obj
    elif isinstance(obj, set):
        return list(obj)
    elif isinstance(obj, dict):
        return {key: convert_to_json_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_json_serializable(item) for item in obj]
    else:
        return str(obj)  # Fallback to string representation


In [12]:
# # Convert the model's dictionary to a JSON-compatible format
json_serializable_model_dict = convert_to_json_serializable(model_dict)
json_serializable_model_str = convert_to_json_serializable(model_str)

In [13]:
# # Save the model's architecture to a JSON file
with open('./model_architecture.json', 'w') as f:
    json.dump(model_config, f, indent=2)

In [14]:
# # Save the model's dictionary to a JSON file
with open('./model_dict.json', 'w') as f:
    json.dump(json_serializable_model_dict, f, indent=4)

In [14]:
# # Save the model's dictionary to a JSON file
# with open('./model_str.json', 'w') as f:
#     json.dump(model_str, f, indent=4)

In [17]:
state_dict = model.state_dict()
embeddings = model.get_input_embeddings()
torch.save(state_dict, './model_state_dict')
torch.save(embeddings.state_dict(), './model_embeddings')

In [18]:
dir(embeddings)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_fill_padding_idx_with_zero',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_is_hf_initialized',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_bac

In [19]:
def get_keys_and_types(state_dict, parent_key='', summaries=None):
    if summaries is None:
        summaries = []
    for key, value in state_dict.items():
        new_key = f"{parent_key}.{key}" if parent_key else key
        if isinstance(value, torch.Tensor):
            summaries.append((new_key, str(value.dtype)))
        elif isinstance(value, dict):
            get_keys_and_types(value, new_key, summaries)
    return summaries

In [20]:
# Get keys and data types
keys_and_types = get_keys_and_types(state_dict)

In [21]:
# Save the keys and data types to a JSON file
with open('./model_structure.json', 'w') as f:
    json.dump(keys_and_types, f, indent=2)

In [22]:
# Print the keys and data types
print(json.dumps(keys_and_types, indent=2))

[
  [
    "embeddings.word_embeddings.weight",
    "torch.float32"
  ],
  [
    "embeddings.position_embeddings.weight",
    "torch.float32"
  ],
  [
    "embeddings.token_type_embeddings.weight",
    "torch.float32"
  ],
  [
    "embeddings.LayerNorm.weight",
    "torch.float32"
  ],
  [
    "embeddings.LayerNorm.bias",
    "torch.float32"
  ],
  [
    "encoder.layer.0.attention.self.query.weight",
    "torch.float32"
  ],
  [
    "encoder.layer.0.attention.self.query.bias",
    "torch.float32"
  ],
  [
    "encoder.layer.0.attention.self.key.weight",
    "torch.float32"
  ],
  [
    "encoder.layer.0.attention.self.key.bias",
    "torch.float32"
  ],
  [
    "encoder.layer.0.attention.self.value.weight",
    "torch.float32"
  ],
  [
    "encoder.layer.0.attention.self.value.bias",
    "torch.float32"
  ],
  [
    "encoder.layer.0.attention.output.dense.weight",
    "torch.float32"
  ],
  [
    "encoder.layer.0.attention.output.dense.bias",
    "torch.float32"
  ],
  [
    "encoder.lay

In [23]:
def format_modules(ordered_dict, indent=0):
    lines = []
    for key, value in ordered_dict.items():
        if isinstance(value, torch.nn.Module):
            lines.append(' ' * indent + f"{key}: {type(value).__name__} ({len(value._modules)} sub-modules)")
            lines.extend(format_modules(value._modules, indent=indent + 2))
        else:
            lines.append(' ' * indent + f"{key}: {type(value).__name__}")
    return lines

In [24]:
model_formatted_string = '\n'.join(format_modules(model._modules))
# Save the model's dictionary to a JSON file
with open('./model_str2.txt', 'w') as f:
    for line in model._modules.__repr__():
        f.write(line)
f.close()
    # json.dump(model_formatted_string, f, indent=4)

In [25]:
dir(model.embeddings)

['LayerNorm',
 'T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_is_hf_initialized',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_m

In [26]:
# Get the word embeddings
word_embeddings = model.embeddings.word_embeddings.weight.data

In [33]:
# Move the embeddings to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
word_embeddings = word_embeddings.to(device)

# Get the tokens and their corresponding indices
vocab = tokenizer.get_vocab()
tokens = [tokenizer.convert_ids_to_tokens(i) for i in range(len(vocab))]

In [34]:
pca = PCA(n_components=2)
word_embeddings_2d = pca.fit_transform(word_embeddings.cpu())

In [35]:
df = pd.DataFrame(word_embeddings_2d, columns=['Semantic Component', 'Syntactic Component'])
df['Token'] = tokens

# Define a color based on token length
df['Color'] = df['Token'].apply(lambda x: len(x))

In [ ]:
# Plot the embeddings using Plotly
fig = px.scatter(df, x='Semantic Component', y='Syntactic Component', color='Color', opacity=0.15)
fig.update_traces(marker=dict(size=5, line=dict(width=0.5, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.update_layout(title='Word Embeddings Visualization', xaxis_title='Semantic Component', yaxis_title='Syntactic Component', showlegend=False, width=800, height=600)

In [29]:
# Load the model and tokenizer
model_name = "bert-base-uncased"
model = BertModel.from_pretrained(model_name, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Move the model to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [37]:
# Get the word embeddings in the model
word_embeddings = model.embeddings.word_embeddings.weight.data.cpu().numpy()

# Perform PCA on the word embeddings
pca = PCA(n_components=2)
word_embeddings_2d = pca.fit_transform(word_embeddings)

In [ ]:
# Create a scatter plot to visualize the word embeddings
fig = go.Figure(data=go.Scattergl(
    x=word_embeddings_2d[:, 0],  # Use the first PCA component as x-axis
    y=word_embeddings_2d[:, 1],  # Use the second PCA component as y-axis
    mode='markers',
    marker=dict(
        color='gray',  # Default color for embeddings
        size=5  # Default size for embeddings
    ),
    opacity=0.07,
    text=[tokenizer.convert_ids_to_tokens(i) for i in range(tokenizer.vocab_size)],  # Use the tokens as text labels
))

In [ ]:
# Prompt
prompt = "mango meade"
prompt_tokens = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
prompt_tokens = {key: value.to(device) for key, value in prompt_tokens.items()}

# Get the attention weights for the prompt
with torch.no_grad():
    prompt_outputs = model(**prompt_tokens)
    attentions = prompt_outputs['attentions']

In [ ]:
# Aggregate and normalize attention weights across all layers and heads
aggregated_attentions = torch.stack(attentions).sum(dim=0).squeeze(0).cpu().numpy()
normalized_attentions = (aggregated_attentions - aggregated_attentions.min()) / (aggregated_attentions.max() - aggregated_attentions.min())

# Sum attention scores across all layers and heads for each token
token_attention_scores = np.sum(normalized_attentions, axis=(0, 1))

In [ ]:
# Highlight the word embeddings that pass through the attention mechanism
highlighted_tokens = set()
for i, attention_score in enumerate(token_attention_scores):
    if attention_score > 0:  # Check if the attention score for this token is greater than 0
        highlighted_tokens.add(i)

In [ ]:
# Update marker color and size for highlighted word embeddings
for i in highlighted_tokens:
    fig.add_trace(go.Scattergl(
        x=[word_embeddings_2d[i, 0]],
        y=[word_embeddings_2d[i, 1]],
        mode='markers',
        marker=dict(color='red', size=10),
        text=tokenizer.convert_ids_to_tokens(i)
    ))

In [ ]:
# Update layout and show plot
fig.update_layout(title='BERT Word Embeddings Visualization with Attention Highlight', xaxis_title='PCA Component 1', yaxis_title='PCA Component 2')
fig.show()